In [1]:
from wxpy import Bot
from sqlalchemy import create_engine
import pandas as pd

def update_sendflag(nick_name, remark_name, value):
    sql = f"""
    UPDATE
        friend_new
    SET
        SendFlag = {value}
    WHERE
        NickName = '{nick_name}'
        AND RemarkName = '{remark_name}'
    """
    conn.execute(sql)

def send_msg(msg, **keys):
    friend = bot.search(**keys)
    if friend:
        friend[0].send_msg(msg)
        return 1
    else:
        return 0
    
bot = Bot(cache_path=True, console_qr=-2)

In [2]:
# 获取微信好友信息
friends = bot.friends()
wx_frnd = pd.DataFrame((each.raw for each in friends))

# 用sqlalchemy读取数据库
engine = create_engine('sqlite:///weixin_friends.sqlite')
db_frnd = pd.read_sql('friend_new', engine) # friend 是旧表，数据未更新

# 合并
df = pd.merge(wx_frnd, 
              db_frnd[['NickName', 'RemarkName', 'Alias', 'SendFlag']], 
              on=['NickName', 'RemarkName'])

print(df.shape)

(810, 38)


In [3]:
filter_1 = df.Alias_y!='skip' # 不跳过
filter_2 = df.SendFlag!=1 # 未发送

df_ = df[filter_1&filter_2]
df_.shape

(786, 38)

In [5]:
# 开始发送啦
from time import sleep

conn = engine.connect()

for idx, row in df_.iterrows():
    Alias = row.Alias_y if row.Alias_y!='null' else '你'
    NickName = row.NickName
    RemarkName = row.RemarkName
    
    msg_text = f'祝原{Alias}新的一年万事顺遂，平安喜乐！'
    value = send_msg(msg_text, nick_name=NickName, remark_name=RemarkName)
    update_sendflag(NickName, RemarkName, value)
    
    info = f"【{idx}】{RemarkName}({NickName})：{msg_text}"
    print(info.ljust(100), end='\r')
    
    sleep(2)
            
conn.close()

KeyboardInterrupt: 

In [ ]:
# !!!仅必要时使用。初始化数据库数据

# 拼合！！！ 这里代码要修改
df = pd.merge(wx_frnd, db_frnd, how='left', left_on='NickName', right_on='nick_name')
df['Alias'] = df['call_name']
df['SendFlag'] = 0
df = df[['UserName','NickName','RemarkName','Sex','City','Province','Signature', 'Alias', 'SendFlag']]

# 选删除，再插入
with engine.connect() as conn:
    conn.execute('DROP TABLE friend_new')
df.to_sql('friend_new', engine, index=False)